In [1]:
# @title 💻 1.1 Check CUDA and Python Version (tested with CUDA 12.2 + Python 3.10.12)

# This notebook is supposed to be connected to NVIDIA drivers with CUDA.
# If this doesn't load check that GPU is selected as hardware accelerator under Edit -> Notebook settings.
!nvidia-smi
!python -V

Sat Mar 29 14:11:32 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        Off | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              N/A / ERR! |      6MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:


from IPython.display import display
import ipywidgets as widgets
import shutil

ckpt_dic = {
    "Original (higher quality)": "prs-eth/marigold-v1-0",
    "LCM (faster)": "prs-eth/marigold-lcm-v1-0",
}

ckpt_name = 'LCM (faster)'
ckpt_path = ckpt_dic[ckpt_name]
w = widgets.Dropdown(
    options=['Original (higher quality)', 'LCM (faster)'],
    value=ckpt_name,
    description='Checkpoint:',
)


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        ckpt_name = change['new']
        ckpt_path = ckpt_dic[ckpt_name]
        # print(f"Change to checkpoint: {ckpt_name} ({ckpt_path})")

w.observe(on_change)

display(w)
# print(f"{ckpt_path = }")

Dropdown(description='Checkpoint:', index=1, options=('Original (higher quality)', 'LCM (faster)'), value='LCM…

In [3]:
print("hello")

hello


In [4]:
# @title 🔧 iii. Setup directories

import os

# Directories
repo_dir = "./" 
input_dir = os.path.join(repo_dir, "input")
output_dir = os.path.join(repo_dir, "output")
output_dir_color = os.path.join(output_dir, "depth_colored")
output_dir_tif = os.path.join(output_dir, "depth_bw")
output_dir_npy = os.path.join(output_dir, "depth_npy")

os.chdir(repo_dir)



In [6]:
# @title 🏇 iv. Initialize pipeline

# change model cache directory
# !export HF_HOME=/content/Marigold/checkpoint

from marigold import MarigoldPipeline

pipe = MarigoldPipeline.from_pretrained(ckpt_path)

pipe = pipe.to("cuda")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:  28%|##8       | 94.4M/335M [00:00<?, ?B/s]

model.safetensors:   3%|3         | 41.9M/1.36G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   2%|2         | 73.4M/3.46G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/d6/7c/d67cd18ed37155aaa23193ef7693cd8616c82eeea39a94eddcfee038eb671f0a/b29dbc05cd2af1ed20492e0c84b323747231dd7bbe52b5f7309f5ecc487e4153?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27diffusion_pytorch_model.safetensors%3B+filename%3D%22diffusion_pytorch_model.safetensors%22%3B&Expires=1743250730&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzI1MDczMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Q2LzdjL2Q2N2NkMThlZDM3MTU1YWFhMjMxOTNlZjc2OTNjZDg2MTZjODJlZWVhMzlhOTRlZGRjZmVlMDM4ZWI2NzFmMGEvYjI5ZGJjMDVjZDJhZjFlZDIwNDkyZTBjODRiMzIzNzQ3MjMxZGQ3YmJlNTJiNWY3MzA5ZjVlY2M0ODdlNDE1Mz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=cC33rIMMD6WXYNIIE1yPdzSqYKXr43AfI75Wtjqpz33TLymHGDh%7ERMQB3e9HCF0BJ2%7ErNdOua4Ga2Tpn9QEhuVaBhlKDZxPiCmi7-MtGuUMN4h3M-UTOnKO5Pxqg7-BX9ug4KObJMckk7A67H3-7Sw0Y-rYWGEtqFhFl24bP%7EgUX8OuhvXphCdCiH%7EzSGdwUGCH%7EucuIVD71SN91OLi5Hl

KeyboardInterrupt: 

In [ ]:
import zipfile

# Path to the zip file in your Google Drive
zip_file_path = '/content/drive/My Drive/datasets/hypersim_processed.zip'

# Destination folder where you want to unzip the files
unzip_path = '/content/hypersim_processed/'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

# Now you can load the dataset from the unzipped directory
import os
print(os.listdir(unzip_path))


['hypersim_processed']


In [ ]:
%%shell

mv /content/hypersim_processed/hypersim_processed/val/filename_list_val.txt /content/Marigold/data_split/hypersim/filename_list_val_filtered.txt

In [ ]:
%%shell

export BASE_DATA_DIR="/content/hypersim_processed"

bash script/eval/61_infer_hypersim.sh

+ ckpt=prs-eth/marigold-v1-0
+ subfolder=eval
+ python infer.py --checkpoint prs-eth/marigold-v1-0 --seed 1234 --base_data_dir /content/hypersim_processed --denoise_steps 50 --ensemble_size 5 --processing_res 512 --dataset_config config/dataset/data_hypersim_val.yaml --output_dir output/eval/hypersim_val/prediction
2025-03-05 07:08:13.622256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741158493.653347   26921 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741158493.665062   26921 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
arguments: Namespace(checkpoint='prs-eth/marigold-v1-0', dataset_config='config/dataset/data_hypersim_val.yaml', base_da

In [ ]:
%%shell
export BASE_DATA_DIR="/content/hypersim_processed"

bash script/eval/62_eval_hypersim.sh

+ subfolder=eval
+ python eval.py --base_data_dir /content/hypersim_processed --dataset_config config/dataset/data_hypersim_val.yaml --alignment least_square --prediction_dir output/eval/hypersim_val/prediction --output_dir output/eval/hypersim_val/eval_metric
Evaluating: 100% 200/200 [00:24<00:00,  8.04it/s]


In [ ]:
!zip -r hypersim_val50.zip /content/Marigold/output/eval/hypersim_val


  adding: content/Marigold/output/eval/hypersim_val/ (stored 0%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ (stored 0%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/ (stored 0%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0014.npy (deflated 13%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0048.npy (deflated 16%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0034.npy (deflated 14%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0091.npy (deflated 12%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0054.npy (deflated 14%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0039.npy (deflated 12%)
  adding: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0097.npy (de

In [ ]:
!dir


config		 doc			   infer.py	README.md	    run.py
CONTRIBUTING.md  environment.yaml	   LICENSE.txt	requirements++.txt  script
data_split	 eval.py		   marigold	requirements+.txt   src
dir.zip		 hypersim_val_metrics.zip  output	requirements.txt    train.py


In [ ]:
!cp hypersim_val50.zip /content/drive/MyDrive/


In [ ]:
!zip -r hypersim_val.zip /content/Marigold/output/eval/hypersim_val/

from google.colab import files
files.download("/content/Marigold/hypersim_val.zip")

updating: content/Marigold/output/eval/hypersim_val/ (stored 0%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ (stored 0%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/ (stored 0%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0014.npy (deflated 13%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0048.npy (deflated 16%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0034.npy (deflated 14%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_01_fr0091.npy (deflated 13%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0054.npy (deflated 14%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0039.npy (deflated 12%)
updating: content/Marigold/output/eval/hypersim_val/prediction/ai_004_004/pred_cam_00_fr0097.npy (de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("hello")

hello
